# NLP 중간고사 대체과제 아마존 리뷰 분류기 만들기
2018310412 인공지능융합전공 고준서

#### pandas를 활용하여 csv파일 불러온 후, 처리한다.

1.   로컬에서 할 경우 같은 경로에 있으면 되는데, 여긴 드라이브 연동으로 실행하여야 함
2.   불러온 후 pos, neg로 되어있는 것을 0 1로 처리한다.



In [0]:
import pandas as pd

In [98]:
import nltk
nltk.download("wordnet")
nltk.download("names")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [99]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/train.csv', 'r') as rt:
    df_train = pd.read_csv(rt)
with open('/content/gdrive/My Drive/Colab Notebooks/test.csv', 'r') as et:
    df_test = pd.read_csv(et)

In [0]:
train_text = list(df_train["text"])
train_sent = list(df_train["senti"])
test_text = list(df_test["text"])
test_sent = list(df_test["senti"])

In [0]:
#pos와 neg로 되어있기 때문에 0과 1로 바꿔준다.
for i in range(len(train_sent)):
    if train_sent[i] == "pos":
        train_sent[i] = 0
    else:
        train_sent[i] = 1
        
for i in range(len(test_sent)):
    if test_sent[i] == "pos":
        test_sent[i] = 0
    else:
        test_sent[i] = 1

In [103]:
train_text[1]

'I loved this dress. i kept putting it on trying to figure out how to get my boobs to not fall out all over the place. this is not a dress for larger breasts. it is absolutely beautiful otherwise!'

### 단어 전처리 - 소문자화, lemmatize, 이름 제거 수행
그러나, 이 부분을 하지 않는게 성능이 더 좋아서 이 부분은 만들어놓고 실제로 실행은 하지 않았다.



In [0]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import names
all_names = set(names.words())

def only_alpha(text):
    a = ""
    for i in text:
        if i.isalpha():
            a += i
    return a

def clean_text(text):
    #단어단위 분절이 아닌 것이 들어와야 한다.
    text = text.split(" ")
    ret_doc = []
    for word in text:
        word = only_alpha(word.lower())
        
        if word not in all_names and len(word) > 2:
            ret_doc.append(word)
    return " ".join(ret_doc)

In [0]:
# for i in range(len(train_text)):
#     train_text[i] = clean_text(train_text[i])

# for i in range(len(test_text)):
#     test_text[i] = clean_text(test_text[i])

### 제일 좋은 Naive Bayes 파라미터가 무엇인지 탐색
best parameter를 아래에 삽입하여 모델을 만듬

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC


nb_ = MultinomialNB()
pipeline = Pipeline([
    ("Vec", CountVectorizer(lowercase=True)),
    ("nb", MultinomialNB()),
])
parameters_pipeline = {
    "nb__alpha":(0.001, 0.01, 0.1, 1, 10, 100),
    "nb__fit_prior":(True, False),
    "Vec__max_df":(0.1,0.25,0.5, 1.0, 10, 100),
}

In [105]:
grid_search = GridSearchCV(pipeline, parameters_pipeline, n_jobs=-1, cv = 3)
grid_search.fit(train_text, train_sent)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('Vec',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prepr

In [111]:
grid_search.best_params_

{'Vec__max_df': 1.0, 'nb__alpha': 1, 'nb__fit_prior': False}

In [112]:
grid_search.best_score_

0.9485022338363315

### 실제로 구해진 파라미터를 적용


1.   'nb__alpha': 1
2.   'nb__fit_prior': False
3.   'Vec__max_df': 1.0

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer(lowercase=True, max_df=1.0)
term_docs_train = CV.fit_transform(train_text)
term_docs_test = CV.transform(test_text)

In [109]:
#최적의 결과를 바탕으로 설정해줌
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha = 1, fit_prior=False)
nb.fit(term_docs_train, train_sent)

MultinomialNB(alpha=1, class_prior=None, fit_prior=False)

### 만들어진 모델 평가 -> 최대 95.037%의 정확도, 매크로평균 91%를 기록한다.

In [131]:
from sklearn.metrics import classification_report
def evaluate(test_x, test_y, model):
    predictions = model.predict(test_x)
    print(classification_report(test_y, predictions))
    print("Macro F1 score - {0:0.5f}".format(classification_report(test_y, predictions, output_dict=True)["macro avg"]["f1-score"]))
evaluate(term_docs_test, test_sent, nb)
accuracy = nb.score(term_docs_test, test_sent)

print("accuracy: %0.3f"%(accuracy*100),"%")

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1260
           1       0.80      0.91      0.85       231

    accuracy                           0.95      1491
   macro avg       0.89      0.93      0.91      1491
weighted avg       0.95      0.95      0.95      1491

Macro F1 score - 0.91023
accuracy: 95.037 %


### 이것은 잘 나온 조합 중 하나입니다.

1번조합: CountVectorizer + MultinomialNB = 95.0(정확도) / 91.0 (메크로 F1)

2번조합: TfidfVectorizer + SVC  = 95.5 / 90.9 (메크로 F1)

-> 그외 딥러닝이나 벡터라이징 방식을 교차해서 해보기도 하였지만 위의 두 조합이 가장 좋은 방식이었다.

2번 조합이 정확도 자체는 더 높으나, 우리가 높여야 하는 것은 macro F1-score이다.

두 방식 모두 report상에서 91로 동일해보이나, 소수점까지 계산하게 되면 

1번조합이 91.01% 2번조합이 90.92%로 1번조합이 약간 더 좋다는 것을 확인할 수 있다.


## 결론 - Macro F1 Score = 91.02%